<p style="font-size: 35px; color: violet; text-align: center;">JIRA</p>

<p style="font-size: 22px; color: skyblue; text-align: center;">Primero debes de logearte a la página de Jira antes de ejecutar los comandos</p>

In [40]:
import numpy as np
import pandas as pd 
import requests
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


from bs4 import BeautifulSoup

#Google sheed API
import gspread
from google.oauth2.service_account import Credentials
#from oauth2client.service_account import ServiceAccountCredentials

#Execute shell commands
import subprocess

---

### Google sheets connection

In [42]:
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com
scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials2.json"
#Cargar credenciales
creds = Credentials.from_service_account_file(credentials, scopes=scope)

#Autorizar gspread con las credenciales
client = gspread.authorize(creds)

# ID de la hoja de calculo
spreadsheet_id = "1lDcw9dWniIy7b9pi9DQFQ-pj4UvHUQkr"

# Abrir la hoja de cálculo por ID
spreadsheet = client.open_by_key(spreadsheet_id)

# Seleccionar la hoja específica dentro del archivo (Se puede usar el nombre de la hoja o el índice)
sheet = spreadsheet.worksheet("Octubre")  # Nombre de la hoja

# Leer los datos en un dataframe
data = sheet.get_all_records()
df = pd.DataFrame(data)

# Mostrar las primeras filas del dataframe
print(df.head())

APIError: APIError: [400]: This operation is not supported for this document

#### Go into google sheets

In [32]:

#Abrir la hoja de calculo por URL o por ID
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1lDcw9dWniIy7b9pi9DQFQ-pj4UvHUQkr/edit?gid=1470561836#gid=1470561836") #https://docs.google.com/spreadsheets/d/<ID_de_tu_hoja_de_calculo> #https://docs.google.com/spreadsheets/d/1lDcw9dWniIy7b9pi9DQFQ-pj4UvHUQkr/edit?gid=1470561836#gid=1470561836

#Seleccionar la hoja específica dentro del archivo (Se puede usar el nombre de la hoja o el índice)
sheet = spreadsheet.worksheet("Octubre") #sheet name

#Leer los datos en un dataframe
data = sheet.get_all_records()
df = pd.DataFrame(data)

#Show firsts rows of the dataframe
print(df.head())


APIError: APIError: [400]: This operation is not supported for this document

---

In [17]:
import socket

def check_port_in_use(port, host='127.0.0.1'):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex((host, port)) == 0

port = 9222
host = '127.0.0.1'
if check_port_in_use(port, host):
    print(f"El puerto {port} está en uso en {host}.")
else:
    print(f"El puerto {port} no está en uso en {host}.")

El puerto 9222 está en uso en 127.0.0.1.


## Chrome connection Windows

In [11]:
#Ejecutar el comando en el cmd
command = '"C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\chrome_debug"'
process = subprocess.Popen(command, shell=True)
time.sleep(2)

#Verifield if the port 9222 is open
command_check = "netstat -ano | findstr 9222"
process_check = subprocess.Popen(command_check, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process_check.communicate()
if stdout:
    print("El puerto 9222 está en uso:")
    print(stdout.decode())
else:
    print("El puerto 9222 no está en uso o hubo un error:")
    print(stderr.decode())

#Configurar las opciones de Chrome para conectarse a la instancia Chrome existente
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

#Configuración del controlador de Chrome
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

html = driver.page_source

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\matia\AppData\Local\Temp\ipykernel_23368\980132825.py:2: SyntaxWarning: invalid escape sequence '\P'
  command = '"C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\chrome_debug"'


El puerto 9222 está en uso:
  TCP    127.0.0.1:9222         0.0.0.0:0              LISTENING       12300
  TCP    127.0.0.1:9222         127.0.0.1:50992        FIN_WAIT_2      900
  TCP    127.0.0.1:9222         127.0.0.1:50993        TIME_WAIT       0
  TCP    127.0.0.1:50992        127.0.0.1:9222         CLOSE_WAIT      8372



---

## Explotanto la información

### Header

In [12]:
#Execute the code in this URL: https://jira.globaldevtools.bbva.com/secure/RapidBoard.jspa?rapidView=33300&quickFilter=106472

driver.get("https://jira.globaldevtools.bbva.com/secure/RapidBoard.jspa?rapidView=33300&quickFilter=106472")
time.sleep(3)
#Get from HTML WEB
element_header = driver.find_element(By.XPATH, "//ul[@class='ghx-column-headers']")
element_header = element_header.get_attribute('innerHTML')

#Starting the local
element_haux = element_header.split("<li")

start_id = element_haux[2].find("id=")
end_id = start_id + 11

list_header = ["To Do", "Analysing", "Pending to information", "In progress", "Ready to deploy", "Ready to verify", "Done"]
list_header_ids = []
for i in element_haux[1: ]:
    list_header_ids.append(i[start_id: end_id])

list_headers = dict(zip(list_header, list_header_ids))
print(list_headers)

{'To Do': 'id="281906"', 'Analysing': 'id="281911"', 'Pending to information': 'id="281914"', 'In progress': 'id="281907"', 'Ready to deploy': 'id="282052"', 'Ready to verify': 'id="282060"', 'Done': 'id="281908"'}


### Columns information

In [28]:
days = range(20) #0 to 19
xpaths = [f"//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-{day} ghx-type-10900 ui-draggable ui-draggable-handle']" for day in days]
combined_xpath = "|".join(xpaths)
print(combined_xpath)

//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-0 ghx-type-10900 ui-draggable ui-draggable-handle']|//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-1 ghx-type-10900 ui-draggable ui-draggable-handle']|//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-2 ghx-type-10900 ui-draggable ui-draggable-handle']|//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-3 ghx-type-10900 ui-draggable ui-draggable-handle']|//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-4 ghx-type-10900 ui-draggable ui-draggable-handle']|//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-5 ghx-type-10900 ui-draggable ui-draggable-handle']|//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-paren

In [23]:
# Cantidad de días acumulados. En este caso 10 días

day0 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-0 ghx-type-10900 ui-draggable ui-draggable-handle']"
day1 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-1 ghx-type-10900 ui-draggable ui-draggable-handle']"
day2 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-2 ghx-type-10900 ui-draggable ui-draggable-handle']"
day3 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-3 ghx-type-10900 ui-draggable ui-draggable-handle']"
day4 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-4 ghx-type-10900 ui-draggable ui-draggable-handle']"
day5 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-5 ghx-type-10900 ui-draggable ui-draggable-handle']"
day6 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-6 ghx-type-10900 ui-draggable ui-draggable-handle']"
day7 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-7 ghx-type-10900 ui-draggable ui-draggable-handle']"
day8 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-8 ghx-type-10900 ui-draggable ui-draggable-handle']"
day9 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-9 ghx-type-10900 ui-draggable ui-draggable-handle']"
day10 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-10 ghx-type-10900 ui-draggable ui-draggable-handle']"
day11 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-11 ghx-type-10900 ui-draggable ui-draggable-handle']"
day12 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-12 ghx-type-10900 ui-draggable ui-draggable-handle']"
day13 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-13 ghx-type-10900 ui-draggable ui-draggable-handle']"
day14 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-14 ghx-type-10900 ui-draggable ui-draggable-handle']"
day15 = "//div[@class='js-detailview ghx-issue js-issue ghx-has-avatar js-parent-drag ghx-not-rankable ghx-days-15 ghx-type-10900 ui-draggable ui-draggable-handle']"

combined_xpath = f"{day0}|{day1}|{day2}|{day3}|{day4}|{day5}|{day6}|{day7}|{day8}|{day9}|{day10}|{day11}|{day12}|{day13}|{day14}|{day15}"

In [29]:
elements = driver.find_elements(By.XPATH, combined_xpath)
elements_html = []
for element in elements: 
    elements_html.append(element.get_attribute('innerHTML'))

for i in range(len(elements_html)): 
    elements_html[i] = elements_html[i][99: 111]

print(len(elements_html), elements_html)

57 ['OTRSSD-17713', 'OTRSSD-17694', 'OTRSSD-17693', 'OTRSSD-17689', 'OTRSSD-17686', 'OTRSSD-17684', 'OTRSSD-17681', 'OTRSSD-17680', 'OTRSSD-17679', 'OTRSSD-17678', 'OTRSSD-17668', 'OTRSSD-17667', 'OTRSSD-17666', 'OTRSSD-17665', 'OTRSSD-17664', 'OTRSSD-17663', 'OTRSSD-17662', 'OTRSSD-17661', 'OTRSSD-17660', 'OTRSSD-17659', 'OTRSSD-17657', 'OTRSSD-17649', 'OTRSSD-17647', 'OTRSSD-17646', 'OTRSSD-17645', 'OTRSSD-17644', 'OTRSSD-17643', 'OTRSSD-17642', 'OTRSSD-17641', 'OTRSSD-17630', 'OTRSSD-17620', 'OTRSSD-17619', 'OTRSSD-17616', 'OTRSSD-17615', 'OTRSSD-17614', 'OTRSSD-17613', 'OTRSSD-17597', 'OTRSSD-17596', 'OTRSSD-17595', 'OTRSSD-17594', 'OTRSSD-17576', 'OTRSSD-17575', 'OTRSSD-17574', 'OTRSSD-17571', 'OTRSSD-17557', 'OTRSSD-17555', 'OTRSSD-17551', 'OTRSSD-17550', 'OTRSSD-17549', 'OTRSSD-17548', 'OTRSSD-17547', 'OTRSSD-17546', 'OTRSSD-17545', 'OTRSSD-17544', 'OTRSSD-17524', 'OTRSSD-17510', 'OTRSSD-17508']


### Accediento a cada petición

In [15]:
#https://jira.globaldevtools.bbva.com/(browse/OTRSSD-?????)
peticiones_url = []
for peticion in elements_html:
    peticiones_url.append(f"https://jira.globaldevtools.bbva.com/browse/{peticion}")

print(peticiones_url[1])

#Abrir nueva pestana (Acceder a la URL de la petición)
driver.execute_script(f"window.open('{peticiones_url[1]}');")
tabs = driver.window_handles
driver.switch_to.window(tabs[-1])

html = driver.page_source

https://jira.globaldevtools.bbva.com/browse/OTRSSD-17694


In [16]:
description = driver.find_elements(By.XPATH, "//div[@class='user-content-block']")
descriptions = []
for descrip in description:
    descriptions.append(descrip.get_attribute('innerHTML'))

print(len(descriptions), '\n', descriptions)

0 
 []
